# **Project #1: Single-View AR Cube**

In [1]:
import cv2
import matplotlib.pyplot as plt
import json
import numpy as np
%matplotlib tk
import os
import matplotlib
try:
    matplotlib.use("TkAgg")
except Exception:
    pass

# 1. Image Choice and Manual Annotations

In [2]:
image_path = "img1.jpg"
img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(12, 8))
plt.imshow(img_rgb)
plt.title("Click points for annotations")
plt.axis("off")

quad_points = plt.ginput(n=4, timeout=0)   # Collect 4 corners of the quadrilateral
points = plt.ginput(n=8, timeout=0)   # Collect 8 points for 4 lines
plt.close()

lines = [
    (points[0], points[1]),  
    (points[2], points[3]), 
    (points[4], points[5]),  
    (points[6], points[7]) 
]

plt.figure(figsize=(12, 8))
plt.imshow(img_rgb)
colors = ["red", "red", "blue", "blue"]

# Draw quadrilateral
quad_np = np.array(quad_points)
plt.plot(np.append(quad_np[:, 0], quad_np[0, 0]),
         np.append(quad_np[:, 1], quad_np[0, 1]),
         color="yellow", linewidth=2)
plt.scatter(quad_np[:, 0], quad_np[:, 1], color="yellow", s=60, edgecolor='black')
for i, (x, y) in enumerate(quad_points):
    plt.text(x+5, y-5, f"Q{i+1}", color="yellow", fontsize=10, weight='bold')

# Draw each line with its label
for i, (p1, p2) in enumerate(lines):
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], color=colors[i], linewidth=2)
    plt.scatter([p1[0], p2[0]], [p1[1], p2[1]], color=colors[i], s=60, edgecolor='black')
    plt.text(p1[0]+5, p1[1]-5, f"L{i+1}", color=colors[i], fontsize=10, weight='bold')

plt.title("Yellow: Quadrilateral, Red/Blue: Two Directions")
plt.axis("off")
plt.tight_layout()

# Save annotated image and annotation data to JSON
output_image_path = "annotations.png"
plt.savefig(output_image_path, bbox_inches="tight", dpi=300)
print(f"Annotated image saved: {output_image_path}")
plt.show()

annotation_data = {
    "image": image_path,
    "quad_points": quad_points,
    "lines": [{"p1": p1, "p2": p2} for (p1, p2) in lines]
}
with open("annotations.json", "w") as f:
    json.dump(annotation_data, f, indent=4)

Annotated image saved: annotations.png


#  2. Vanishing Points, Horizon, and Intrinsics

In [3]:
with open("annotations.json", "r") as f:
    data = json.load(f)

image_path = data["image"]
quad_points = np.array(data["quad_points"])
lines = data["lines"]

img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = img_rgb.shape[:2]

# Compute line equation ax + by + c = 0 from two points
def line_from_points(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    a = y1 - y2
    b = x2 - x1
    c = x1 * y2 - x2 * y1
    return np.array([a, b, c], dtype=float)

L = [line_from_points(l["p1"], l["p2"]) for l in lines]   # Compute line equations

# Compute vanishing points
v1 = np.cross(L[0], L[1])
v1 = v1 / v1[2]
v2 = np.cross(L[2], L[3])
v2 = v2 / v2[2]

print("\nVanishing Point Estimation:")
print(f"Vanishing Point 1: ({v1[0]:.2f}, {v1[1]:.2f})")
print(f"Vanishing Point 2: ({v2[0]:.2f}, {v2[1]:.2f})")

# Compute horizon line
horizon = np.cross(v1, v2)
if abs(horizon[2]) > 1e-8:
    horizon = horizon / horizon[2]
    x_vals = np.array([0, w])
    y_vals = (-horizon[0] * x_vals - horizon[2]) / horizon[1]
else:
    y_vals = None
    print("Warning: Horizon at infinity (parallel vanishing points)")

# Estimate focal length using orthogonality
cx, cy = w / 2, h / 2
v1x, v1y = v1[0] - cx, v1[1] - cy
v2x, v2y = v2[0] - cx, v2[1] - cy
f_squared = -(v1x * v2x + v1y * v2y)
f = np.sqrt(abs(f_squared))
print(f"Estimated focal length f: {f:.2f} pixels")

# Intrinsic matrix K
K = np.array([[f, 0, cx],
              [0, f, cy],
              [0, 0, 1]], dtype=float)

# Visualize vanishing points, lines, horizon, and principal point
plt.figure(figsize=(12, 8))
plt.imshow(img_rgb)
colors = ["red", "red", "blue", "blue"]

for i, l in enumerate(lines):
    p1, p2 = l["p1"], l["p2"]
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], color=colors[i], linewidth=2)
    plt.scatter([p1[0], p2[0]], [p1[1], p2[1]], color=colors[i], s=50, edgecolor='black')

plt.scatter(v1[0], v1[1], color='orange', s=120, marker='x', linewidths=3, label='Vanishing Point 1')
plt.scatter(v2[0], v2[1], color='cyan',   s=120, marker='x', linewidths=3, label='Vanishing Point 2')

if y_vals is not None:
    plt.plot(x_vals, y_vals, 'g--', linewidth=2, label='Horizon Line')

plt.scatter(cx, cy, color='magenta', s=100, marker='+', linewidths=3, label='Principal Point')
plt.legend(fontsize=10)
plt.title("Vanishing Points, Horizon Line, and Intrinsic Calibration")
plt.axis("off")
plt.tight_layout()

print("\nPrincipal Point:", (cx, cy))
print("Intrinsic Matrix K:\n", K)

# Save visualization and results
output_path = "vanishing_horizon_focal.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"Visualization saved: {output_path}")

results = {
    "vanishing_points": {"v1": v1.tolist()[:2], "v2": v2.tolist()[:2]},
    "horizon_line": horizon.tolist() if y_vals is not None else None,
    "focal_length": float(f),
    "principal_point": [float(cx), float(cy)],
    "K": K.tolist()
}
with open("vanishing_horizon_intrinsics.json", "w") as f:
    json.dump(results, f, indent=4)


Vanishing Point Estimation:
Vanishing Point 1: (1532.99, 783.84)
Vanishing Point 2: (-790.32, 726.76)
Estimated focal length f: 1161.11 pixels

Principal Point: (408.0, 728.0)
Intrinsic Matrix K:
 [[1.16110505e+03 0.00000000e+00 4.08000000e+02]
 [0.00000000e+00 1.16110505e+03 7.28000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Visualization saved: vanishing_horizon_focal.png


#  3. Planar Homography and Pose Recovery

In [4]:
def load_annotations(path="annotations.json"):
    with open(path, "r") as f:
        return json.load(f)

# 2D point normalization for DLT
def normalize_points_2d(pts):
    pts = np.asarray(pts, dtype=float)
    N = pts.shape[0]
    mean = pts.mean(axis=0)
    std = pts.std(axis=0).mean()
    if std < 1e-8:
        std = 1.0
    s = np.sqrt(2) / std
    T = np.array([[s, 0, -s*mean[0]],
                  [0, s, -s*mean[1]],
                  [0, 0, 1]], dtype=float)
    pts_h = np.hstack([pts, np.ones((N,1))])
    pts_n = (T @ pts_h.T).T
    return T, pts_n

# Denormalize homography after DLT
def denormalize_H(Hn, T_src, T_dst):
    return np.linalg.inv(T_dst) @ Hn @ T_src

# Compute homography using Normalized Direct Linear Transformation
def compute_homography_normalized(src_pts, dst_pts):
    src = np.asarray(src_pts, dtype=float)
    dst = np.asarray(dst_pts, dtype=float)
    assert src.shape[0] == dst.shape[0] and src.shape[0] >= 4

    T_src, src_n = normalize_points_2d(src)
    T_dst, dst_n = normalize_points_2d(dst)

    A = []
    for i in range(src_n.shape[0]):
        x, y, _ = src_n[i]
        u, v, _ = dst_n[i]
        A.append([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
        A.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])
    A = np.asarray(A)

    _, _, Vt = np.linalg.svd(A)
    Hn = Vt[-1].reshape(3, 3)
    H = denormalize_H(Hn, T_src, T_dst)
    if abs(H[2, 2]) > 1e-12:
        H /= H[2, 2]
    return H

# Compute reprojection error
def reprojection_error(H, world_pts, image_pts):
    world = np.hstack([world_pts, np.ones((world_pts.shape[0],1))])
    proj = (H @ world.T).T
    proj = proj[:, :2] / proj[:, 2:3]
    diffs = np.linalg.norm(proj - image_pts, axis=1)
    return diffs, proj

if __name__ == "__main__":
    if not os.path.exists("annotations.json"):
        raise FileNotFoundError("Error! annotations.json not found.")

    data = load_annotations("annotations.json")
    image_path = data["image"]
    quad_points = np.asarray(data["quad_points"], dtype=float)

    if quad_points.shape[0] != 4:
        raise ValueError("quad_points must contain exactly 4 points.")

    # Load intrinsics from Section 2    
    if os.path.exists("vanishing_horizon_intrinsics.json"):
        with open("vanishing_horizon_intrinsics.json", "r") as f:
            intrinsics = json.load(f)
        K = np.array(intrinsics["K"], dtype=float)
    else:
        print("\nWarning: Intrinsics file not found!")

    world_pts = np.array([[0,0],[1,0],[1,1],[0,1]], dtype=float)
    H = compute_homography_normalized(world_pts, quad_points)    # Estimate homography
    print("\nEstimated Homography H:\n", H)

    # Reprojection error
    diffs, proj = reprojection_error(H, world_pts, quad_points)
    mean_err = np.mean(diffs)
    print(f"\nReprojection error per corner: {diffs}")
    print(f"Mean reprojection error: {mean_err:.4f}")

    # Pose recovery from H and K
    K_inv = np.linalg.inv(K)
    H_tilde = K_inv @ H
    h1, h2, h3 = H_tilde[:, 0], H_tilde[:, 1], H_tilde[:, 2]

    norm1, norm2 = np.linalg.norm(h1), np.linalg.norm(h2)
    lam = (norm1 + norm2) / 2.0
    r1, r2 = h1 / norm1, h2 / norm2
    r3 = np.cross(r1, r2)
    R_approx = np.column_stack((r1, r2, r3))

    U, _, Vt = np.linalg.svd(R_approx)
    R = U @ Vt
    if np.linalg.det(R) < 0:
        U[:, -1] *= -1
        R = U @ Vt
    t = h3 / lam
    if t[2] < 0:
        R = -R
        t = -t
    print("R:\n", R)
    print("t:\n", t)

    # Visualize pose and reprojection
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb)

    # Original quad points
    plt.scatter(quad_points[:,0], quad_points[:,1], c='yellow', s=80, edgecolor='black', label='Quad Points')
    for i, (x, y) in enumerate(quad_points):
        plt.text(x+4, y-4, f"Q{i+1}", color='yellow', fontsize=10, weight='bold')

    # Reprojected points
    plt.scatter(proj[:,0], proj[:,1], c='white', s=50, marker='x', label='Reprojected')

    # Project world axes
    axis_len = 0.5
    axes_world = np.array([
        [0,0,0],
        [axis_len,0,0],   # X-axis endpoint
        [0,axis_len,0],   # Y-axis endpoint
        [0,0,1]   # Z-axis endpoint
    ], dtype=float)

    # Build projection matrix
    P = K @ np.hstack((R, t.reshape(3,1)))
    axes_proj = []
    for X in axes_world:
        Xh = np.hstack((X, 1))
        x = P @ Xh
        x /= x[2]
        axes_proj.append(x[:2])
    axes_proj = np.array(axes_proj)

    o, px, py, pz = axes_proj
    plt.plot([o[0], px[0]], [o[1], px[1]], 'r', linewidth=3, label='X axis')
    plt.plot([o[0], py[0]], [o[1], py[1]], 'm', linewidth=3, label='Y axis')
    plt.plot([o[0], pz[0]], [o[1], pz[1]], 'b', linewidth=3, label='Z axis')

    plt.legend()
    plt.title(f"Camera Pose Recovery")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig("pose_visualization.png", dpi=300, bbox_inches='tight')
    plt.show()

    # Save estimated parameters
    np.savetxt("H_estimated.txt", H, fmt="%.8f")
    np.savetxt("K_estimated.txt", K, fmt="%.8f")
    np.savetxt("R_estimated.txt", R, fmt="%.8f")
    np.savetxt("t_estimated.txt", t.reshape(1,3), fmt="%.8f")
    print("\nSaved H, R, t to text files.")


Estimated Homography H:
 [[ 1.61265003e+02  1.38888740e+02  2.46772727e+02]
 [-2.61301248e+01 -4.37388592e+00  1.11513636e+03]
 [ 1.30718954e-02 -4.24836601e-02  1.00000000e+00]]

Reprojection error per corner: [0.00000000e+00 5.68434189e-14 3.21554936e-13 0.00000000e+00]
Mean reprojection error: 0.0000
R:
 [[ 0.74047285  0.66512632  0.09647249]
 [-0.51564531  0.47016678  0.71627726]
 [ 0.43105669 -0.58012945  0.691115  ]]
t:
 [-0.98719931  2.37044728  7.10948019]

Saved H, R, t to text files.


#  4. Cube Definition, Projection, and Rendering

In [5]:
IMAGE_PATH = "img1.jpg"
OUT_WIREFRAME = "cube_wireframe.png"
OUT_SHADED = "cube_shaded.png"
s = 1.0

def load_matrix_txt(path):
    if not os.path.exists(path):
        return None
    return np.loadtxt(path)

# Project 3D points to 2D image
def project_points(P, X):
    N = X.shape[0]
    Xh = np.hstack([X, np.ones((N,1))])  
    x = (P @ Xh.T).T  
    zs = ( (P @ Xh.T).T )[:,2]           
    valid = np.abs(x[:,2]) > 1e-9
    uv = np.zeros((N,2))
    uv[valid] = (x[valid,:2].T / x[valid,2]).T
    return uv, zs, valid

# Build cube aligned to support plane
def build_cube(base_origin=(0.0,0.0), s=1.0):
    x0,y0 = base_origin
    pts = [
        [x0,     y0,     0.0],  
        [x0 + s, y0,     0.0],  
        [x0 + s, y0 + s, 0.0],  
        [x0,     y0 + s, 0.0],  
        [x0,     y0,     s],    
        [x0 + s, y0,     s],   
        [x0 + s, y0 + s, s],   
        [x0,     y0 + s, s],   
    ]
    return np.array(pts, dtype=float)

# Draw wireframe cube overlay
def draw_wireframe_on_image(img_rgb, proj_uv, edges, filename):
    fig, ax = plt.subplots(figsize=(8,10))
    ax.imshow(img_rgb)
    ax.axis('off')
    for (i,j) in edges:
        p1 = proj_uv[i]; p2 = proj_uv[j]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color='lime', linewidth=2)

    ax.scatter(proj_uv[:,0], proj_uv[:,1], c='yellow', s=40, edgecolors='black')
    plt.tight_layout()
    fig.savefig(filename, dpi=150, bbox_inches='tight')
    plt.close(fig)

# Draw shaded cube with depth ordering
def draw_shaded(img_rgb, proj_uv, face_indices, depths, face_colors, filename):
    face_depths = []
    for f in face_indices:
        face_depths.append(np.mean(depths[list(f)]))
    order = np.argsort(face_depths)[::-1]  

    fig, ax = plt.subplots(figsize=(8,10))
    ax.imshow(img_rgb)
    ax.axis('off')

    # Fill faces with corresponding colors
    for idx in order:
        f = face_indices[idx]
        poly = np.array([proj_uv[i] for i in f])
        ax.fill(poly[:,0], poly[:,1], face_colors[idx], alpha=0.8, edgecolor='black', linewidth=1.5)

    # Draw cube edges
    for f in face_indices:
        poly = np.array([proj_uv[i] for i in f])
        ax.plot(np.append(poly[:,0], poly[0,0]), np.append(poly[:,1], poly[0,1]), color='black', linewidth=1.0)

    ax.scatter(proj_uv[:,0], proj_uv[:,1], c='yellow', s=30, edgecolors='black')
    plt.tight_layout()
    fig.savefig(filename, dpi=150, bbox_inches='tight')
    plt.close(fig)

if __name__ == "__main__":
    if not os.path.exists(IMAGE_PATH):
        raise FileNotFoundError(f"Image not found: {IMAGE_PATH}")
    img = cv2.imread(IMAGE_PATH)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    K = load_matrix_txt("K_estimated.txt")
    R = load_matrix_txt("R_estimated.txt")
    t = load_matrix_txt("t_estimated.txt")
    if K is None or R is None or t is None:
        raise FileNotFoundError("K_estimated.txt, R_estimated.txt, or t_estimated.txt not found!")

    t = t.reshape(3,)
    P = K @ np.hstack((R, t.reshape(3,1)))
    # Build and project cube
    cube_world = build_cube(base_origin=(0.0, 0.0), s=s) 
    proj_uv, raw_z, valid = project_points(P, cube_world)
    Z_cam = (R @ cube_world.T + t.reshape(3,1))[2,:]

    unstable = False   # Stability check
    msg = []
    for i, z in enumerate(Z_cam):
        if z <= 1e-6:
            unstable = True
            msg.append(f"Vertex {i} has non-positive camera z = {z:.6e}")
        if not np.isfinite(proj_uv[i,0]) or not np.isfinite(proj_uv[i,1]) or abs(proj_uv[i,0])>1e6 or abs(proj_uv[i,1])>1e6:
            unstable = True
            msg.append(f"Vertex {i}: unstable projection {proj_uv[i]}.")

    if unstable:
        print("\nSome vertices showed instability!")

    # Define cube faces and base colors
    faces = [
        [0,1,2,3],  
        [4,5,6,7],  
        [0,1,5,4], 
        [1,2,6,5],  
        [2,3,7,6],  
        [3,0,4,7]  
    ]

    face_colors = [
        (0.9,0.9,0.9),  
        (0.7,0.7,0.9),  
        (0.9,0.6,0.6),  
        (0.9,0.8,0.6),  
        (0.8,0.7,0.9),  
        (0.9,0.9,0.6)   
    ]

    # Wireframe rendering
    draw_wireframe_on_image(img_rgb, proj_uv, edges=[(0,1),(1,2),(2,3),(3,0),
                                                     (4,5),(5,6),(6,7),(7,4),
                                                     (0,4),(1,5),(2,6),(3,7)],
                            filename=OUT_WIREFRAME)
    print(f"Saved wireframe overlay: {OUT_WIREFRAME}")

    depths = Z_cam.copy()  
    shaded_colors = []
    cam_pts = (R @ cube_world.T + t.reshape(3,1)).T  
    light_dir = np.array([0.0, 0.0, 1.0])  
    light_dir = light_dir / np.linalg.norm(light_dir)

    for f in faces:
        p0 = cam_pts[f[0]]; p1 = cam_pts[f[1]]; p2 = cam_pts[f[2]]
        v1 = p1 - p0; v2 = p2 - p0
        n = np.cross(v1, v2)
        n_norm = np.linalg.norm(n)
        if n_norm < 1e-9:
            shade = 0.5
        else:
            n = n / n_norm
            shade = np.clip(np.dot(n, light_dir), 0.0, 1.0) * 0.7 + 0.3
        base_color = np.array(face_colors[faces.index(f)])
        shaded_colors.append(tuple((base_color * shade).tolist()))

    # Shaded rendering with depth ordering
    draw_shaded(img_rgb, proj_uv, face_indices=faces, depths=depths, face_colors=shaded_colors, filename=OUT_SHADED)
    print(f"Saved shaded faces image: {OUT_SHADED}")
    # print(f"Projected cube vertices:\n{proj_uv}")
    # print(f"\nCamera-space Z of vertices:\n{Z_cam}")

Saved wireframe overlay: cube_wireframe.png
Saved shaded faces image: cube_shaded.png


#  5. Face Shading Implementation

In [6]:
# Transform cube vertices into camera coordinates
cam_pts = (R @ cube_world.T + t.reshape(3,1)).T  
light_dir = np.array([0.0, 0.0, 1.0])          
light_dir = -light_dir / np.linalg.norm(light_dir)  

shaded_colors = []
shading_report = []
face_names = ["bottom", "top", "front", "right", "back", "left"]

# Compute per-face shading
for idx, f in enumerate(faces):
    v0 = cam_pts[f[0]]
    v1 = cam_pts[f[1]]
    v2 = cam_pts[f[2]]
    
    n = np.cross(v1 - v0, v2 - v0)
    n_len = np.linalg.norm(n)
    
    if n_len < 1e-9:
        brightness = 0.2
        normal_hat = [0.0, 0.0, 0.0]
    else:
        normal_hat = n / n_len
        dot = np.dot(normal_hat, light_dir)   # n̂ · (-l)   
        brightness = max(0.2, dot)               
    
    # Apply brightness to base color
    base_color = np.array(face_colors[idx])
    final_color = tuple((base_color * brightness).tolist())
    shaded_colors.append(final_color)

    # Record shading data for report
    shading_report.append({
        "face_index": idx,
        "face_name": face_names[idx],
        "vertices": [int(f[0]), int(f[1]), int(f[2])],
        "normal_vector": [round(x, 6) for x in normal_hat.tolist()],
        "brightness": round(brightness, 6),
        "base_color": face_colors[idx],
        "final_color": final_color
    })

with open("shading_report.json", "w") as f:
    json.dump({"faces": shading_report}, f, indent=2)
print("Saved shading report: shading_report.json")

Saved shading report: shading_report.json
